In [65]:
interp1d(range(len(a), fp, fill_value="extrapolate")

<class 'numpy.ndarray'>


In [42]:
import numpy as np
from skimage.transform import resize

a = np.array([[1, 2, 4], [0.0, 0, 0]])

# Use order=1 for linear interpolation
result = resize(a, (a.shape[0] * 2, a.shape[1] * 2), order=1, mode="edge", anti_aliasing=False)
# The result will no longer have tiny artifact values.
print(np.round(result, 2))

[[1.   1.25 1.75 2.5  3.5  4.  ]
 [0.75 0.94 1.31 1.88 2.62 3.  ]
 [0.25 0.31 0.44 0.62 0.88 1.  ]
 [0.   0.   0.   0.   0.   0.  ]]


In [ ]:
ds_pangu[shortName].plot.hist()

ds_pangu[shortName].mean().compute()

ds_pangu[shortName].min(), ds_pangu[shortName].max()

z = ds_pangu["z"]
# --- Please run this exact block of code with your DataArray 'z' ---

# 1. Print the full representation of your object 'z'
# This will show its name, dimensions, coordinates, and attributes.
print("--- Full representation of the DataArray 'z' ---")
print(z)
print("-------------------------------------------------")


# 2. Calculate all three statistics back-to-back
min_val = z.min()
max_val = z.max()
mean_val = z.mean()

# 3. Print the results
print(f"Min value from .min(): {min_val.item()}")
print(f"Max value from .max(): {max_val.item()}")
print(f"Mean value from .mean(): {mean_val.item()}")


# 4. Perform the crucial consistency check
is_consistent = min_val <= mean_val <= max_val

print("\n--- Consistency Check ---")
print(f"Is the result mathematically consistent? {is_consistent}")
print("-------------------------")

# This forces a true NumPy calculation on the in-memory data,
# bypassing any xarray/cfgrib backend magic.
true_mean = z.values.mean()

print(f"The TRUE mathematical mean is: {true_mean}")

z.attrs

# del(z.attrs['GRIB_paramId'])
# del(z.attrs['GRIB_cfName'])
# del(z.attrs['GRIB_cfVarName'])
# del(z.attrs['GRIB_name'])
# del(z.attrs['GRIB_shortName'])
z.attrs.update({"long_name": "Geopotential Height", "standard_name": "geopotential height"})
z.mean()

ds_fengwu[shortName].plot.hist()

ds_gefs[shortName].plot.hist()

In [49]:
# reran fengwu to get values. they were all zero at first.
initialization_time = pd.Timestamp("20240526")
mem = "p29"
ifiles = Path(
    f"/glade/derecho/scratch/ahijevyc/ai-models/output/fengwu/{initialization_time:%Y%m%d%H}/{mem}"
).glob("f*.nc")
junk = xr.open_mfdataset(ifiles, decode_timedelta=True)
if shortName == "z":
    junk = junk / g.m

prediction_timedelta = pd.to_timedelta(np.arange(1, 11), unit="D")

fixed_values = junk.__xarray_dataarray_variable__.sel(
    channel=f"{shortName}{isobaricInhPa}", prediction_timedelta=prediction_timedelta
).load()

target_labels = dict(number=int(mem[1:]), initialization_time=initialization_time)

ds_fengwu[shortName].loc[target_labels] = fixed_values.squeeze().values

In [50]:
ofile = CACHEDIR / f"fengwu.{isobaricInhPa}{shortName}.{years}.{forecast_length:03d}h.zarr2"

ds_fengwu.to_zarr(ofile)

/glade/work/ahijevyc/conda-envs/daa/lib/python3.12/site-packages/zarr/core/dtype/npy/string.py:248: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=4, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without warning in a future version of Zarr Python. Arrays stored with this data type may be unreadable by other Zarr libraries. Use this data type at your own risk! Check https://github.com/zarr-developers/zarr-extensions/tree/main/data-types for the status of data type specifications for Zarr V3.
  v3_unstable_dtype_warning(self)
/glade/work/ahijevyc/conda-envs/daa/lib/python3.12/site-packages/zarr/api/asynchronous.py:229: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


In [10]:
# they were all zero at first.
ifile = Path("/glade/derecho/scratch/ahijevyc/tmp/fengwu_gefs_pred_048.nc")
junk = xr.open_mfdataset(ifile)
junk.__xarray_dataarray_variable__.sel(channel="t850").load()

<xarray.DataArray '__xarray_dataarray_variable__' (init_time: 1,
                                                   prediction_timedelta: 1,
                                                   lat: 161, lon: 321)> Size: 413kB
array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]])
Coordinates:
  * init_time             (init_time) datetime64[ns] 8B 2024-05-29
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 8B 2 days
    channel               <U4 16B 't850'
  * lat                   (lat) float64 1kB 60.0 59.75 59.5 ... 20.5 20.25 20.0
  * lon                   (lon) float64 3kB 220.0 220.2 220.5 ... 299.8 300.0

In [ ]:
# instead of recreating the zarr, just load the existing one and drop the extra init times and resave it with .zarr2 extension
# Then rename the old one with _hide suffix and rename the .zarr2 extension .zarr.
times_to_drop = pd.date_range("2023-06-01", "2023-06-07").union(
    pd.date_range("2024-06-01", "2024-06-07")
)
ds_gefs = ds_gefs.drop_sel(initialization_time=times_to_drop)
ds_gefs

ds_gefs.to_zarr(ofile.with_suffix(".zarr2"))